<a href="https://colab.research.google.com/github/arielwendichansky/DI_Bootcamp/blob/master/Week7/Day3/Exercise_XP/Scraping_Data_From_A_Dynamic_Webpage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install selenium

In [117]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

Initialize Selenium WebDriver

Extract necessary data such as plan names, features, and pricing

In [141]:
# Configure the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless to avoid UI pop-up
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

# Target URL
url = "https://www.inmotionhosting.com/"
# Navigate to the page
driver.get(url)


# Wait for the "Shared Hosting Plans" link to be clickable
wait = WebDriverWait(driver, 10)  # Maximum wait time of 10 seconds
pop_up = driver.find_element(By.ID, 'onetrust-close-btn-container')
pop_up.click()
# Find the element by link text
link = driver.find_element(By.LINK_TEXT, 'Shared Hosting Plans')
# Click on the link
link.click()

try:
    wait = WebDriverWait(driver, 10)
    # Wait for the element to be present in the DOM
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'imh-spec-table-container')))

    # Get page source and prepare it for parsing
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the desired element within the element found by Selenium
    name_list = soup.find('thead', class_="desktop-view").find_all('span',class_="column-title")
    # It's saved as a list and then splitted for names
    name = [name.text.strip() for name in name_list]
    # Find the desired prices within the element found by Selenium
    price_list = soup.find('div', class_="imh-rostrum-container").find_all('div',class_="imh-rostrum-starting-at-price-discounted")
    # It's saved as a list and then splitted for prices
    price = [price.text.strip() for price in price_list]

    #Find the desired features name within the element found by Selenium
    features_name = soup.find('tbody', class_="desktop-view").find_all('td',class_='tooltips-table')
    feature = [feature.text.strip() for feature in features_name]

    #Find the desired features values within the element found by Selenium
    features_value = soup.find('tbody', class_="desktop-view").find_all('td')[1:]

    values_data = [value.text.strip() for value in features_value]
    values = [value for value in values_data if value not in feature]

    # for i in values:
    #   if i not in features_name:
    #     values = values.remove(i)

    print(name)
    print(price)
    print(feature)
    print(values)
finally:
    driver.quit()

['Core', 'Launch', 'Power', 'Pro']
['$2.49/mo', '$2.99/mo', '$3.49/mo', '$4.99/mo', '$5.99/mo', '$6.99/mo', '$8.99/mo', '$9.99/mo', '$10.99/mo', '$12.99/mo', '$13.99/mo', '$14.99/mo', '$19.99/mo']
['Websites Supported', 'Disk Space', 'Bandwidth', 'Email Accounts', 'Dedicated IP Address', 'No-Downtime Website Migration', 'UltraStack Optimized Performance', 'Free Domain Name', 'Free SSL', 'eCommerce Ready', 'Pro Level Support', 'Advanced Caching', 'BoldGrid Website Builder', 'Parked Domains', 'Sub Domains', 'MySQL & PostgreSQL Databases', 'Email Storage per Inbox', 'Hosting Plus', 'Choice of Data Center', 'Phone, Chat and Ticket Support']
['2 Websites', '25 Websites', 'Unlimited Websites', 'Unlimited Websites', '100 GB SSD', 'Unlimited NVMe SSDs', 'Unlimited NVMe SSDs', 'Unlimited NVMe SSDs', 'Unlimited', 'Unlimited', 'Unlimited', 'Unlimited', '10 Email Addresses', 'Unlimited Email Addresses', 'Unlimited Email Addresses', 'Unlimited Email Addresses', 'Available', 'Available', 'Available'

Store and Save the Data


In [76]:
# Organize prices by years for each name
prices_by_years = {
    'Core': price[:3],
    'Launch': price[3:6],
    'Power': price[6:9],
    'Pro': price[9:12]
}

# Create a DataFrame from the dictionary
df_per_year = pd.DataFrame(prices_by_years)

# Set the index to represent years
df_per_year.index = ['3-years', '2-years', '1-year']

# Print the DataFrame
print(df_per_year)


             Core    Launch      Power        Pro
3-years  $2.49/mo  $4.99/mo   $8.99/mo  $12.99/mo
2-years  $2.99/mo  $5.99/mo   $9.99/mo  $13.99/mo
1-year   $3.49/mo  $6.99/mo  $10.99/mo  $14.99/mo


In [77]:
pro_1_month = {'Pro': price[12:13]}
df_per_month = pd.DataFrame(pro_1_month)
df_per_month.index = ['1-month']
print(df_per_month)

               Pro
1-month  $19.99/mo


In [160]:
df_feature = pd.DataFrame(columns=name)
df_feature

,Core,Launch,Power,Pro


In [133]:
for i, value in enumerate(values_data):
  length = len(df_feature)
  df_feature.loc[length]= i

df_feature

,Features,Core,Launch,Power,Pro
0,Websites Supported,NaN,NaN,NaN,NaN
1,NaN,2 Websites,NaN,NaN,NaN
2,NaN,NaN,25 Websites,NaN,NaN
3,NaN,NaN,NaN,Unlimited Websites,NaN
4,NaN,NaN,NaN,NaN,Unlimited Websites
...,...,...,...,...,...
695,95,95,95,95,95
696,96,96,96,96,96
697,97,97,97,97,97
698,98,98,98,98,98


In [159]:
import pandas as pd

# Example lists
column_names = name
index_names = features_name
values = values
sublists = [values[i:i+4] for i in range(0, len(values), 4)]

# print(df)
print(sublists)

[['2 Websites', '25 Websites', 'Unlimited Websites', 'Unlimited Websites'], ['100 GB SSD', 'Unlimited NVMe SSDs', 'Unlimited NVMe SSDs', 'Unlimited NVMe SSDs'], ['Unlimited', 'Unlimited', 'Unlimited', 'Unlimited'], ['10 Email Addresses', 'Unlimited Email Addresses', 'Unlimited Email Addresses', 'Unlimited Email Addresses'], ['Available', 'Available', 'Available', ''], ['Available', 'Available', 'Available', 'Available'], ['2x Speed and Performance', '6X UltraStack Speed & Performance', '12X UltraStack Speed & Performance', '20X UltraStack Speed & Performance'], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['1', 'Unlimited', 'Unlimited', 'Unlimited'], ['Unlimited', 'Unlimited', 'Unlimited', 'Unlimited'], ['40', 'Unlimited', 'Unlimited', 'Unlimited'], ['1 GB', '5GB', '10 GB', '20 GB'], ['', '', '', ''], ['', '', '', ''], ['Chat and Ticket Only', 'Chat and Ticket Only', '', '']]


In [164]:
for value in sublists:
  length = len(df_feature)
  df_feature.loc[length]= value

df_feature.index = feature
df_feature

,Core,Launch,Power,Pro
Websites Supported,2 Websites,25 Websites,Unlimited Websites,Unlimited Websites
Disk Space,100 GB SSD,Unlimited NVMe SSDs,Unlimited NVMe SSDs,Unlimited NVMe SSDs
Bandwidth,Unlimited,Unlimited,Unlimited,Unlimited
Email Accounts,10 Email Addresses,Unlimited Email Addresses,Unlimited Email Addresses,Unlimited Email Addresses
Dedicated IP Address,Available,Available,Available,
No-Downtime Website Migration,Available,Available,Available,Available
UltraStack Optimized Performance,2x Speed and Performance,6X UltraStack Speed & Performance,12X UltraStack Speed & Performance,20X UltraStack Speed & Performance
Free Domain Name,,,,
Free SSL,,,,
eCommerce Ready,,,,
